# Exercise 14 - Multi-Resolution Feature Extraction with PointNet++

This notebook shows training and prediction of PointNet++ for aerial point clouds. Its implementation is a simplified version based on the implementation of Keras layers that can be found on https://github.com/dgriffiths3/pointnet2-tensorflow2. This notebook is rather rough with few comments, as most was already explained in previous notebooks and the code studies of the PointNet++ layers provided in additional notebooks.

There are no active tasks for this course topic, just a demonstration to study, train, and do some predictions with PointNet++.

# Setup TensorFLow

In [1]:
# Change X to the GPU number you want to use,
# otherwise you will get a Python error
# e.g. USE_GPU = 4
USE_GPU = X

In [2]:
# Import TensorFlow 
import tensorflow as tf

# Print the installed TensorFlow version
print(f'TensorFlow version: {tf.__version__}\n')

# Get all GPU devices on this server
gpu_devices = tf.config.list_physical_devices('GPU')

# Print the name and the type of all GPU devices
print('Available GPU Devices:')
for gpu in gpu_devices:
    print(' ', gpu.name, gpu.device_type)
    
# Set only the GPU specified as USE_GPU to be visible
tf.config.set_visible_devices(gpu_devices[USE_GPU], 'GPU')

# Get all visible GPU  devices on this server
visible_devices = tf.config.get_visible_devices('GPU')

# Print the name and the type of all visible GPU devices
print('\nVisible GPU Devices:')
for gpu in visible_devices:
    print(' ', gpu.name, gpu.device_type)
    
# Set the visible device(s) to not allocate all available memory at once,
# but rather let the memory grow whenever needed
for gpu in visible_devices:
    tf.config.experimental.set_memory_growth(gpu, True)
    
# Import Keras
from tensorflow import keras

# Print the installed Keras version
print(f'\nKeras version: {keras.__version__}\n')

TensorFlow version: 2.3.1

Available GPU Devices:
  /physical_device:GPU:0 GPU
  /physical_device:GPU:1 GPU
  /physical_device:GPU:2 GPU
  /physical_device:GPU:3 GPU
  /physical_device:GPU:4 GPU
  /physical_device:GPU:5 GPU
  /physical_device:GPU:6 GPU
  /physical_device:GPU:7 GPU

Visible GPU Devices:
  /physical_device:GPU:4 GPU

Keras version: 2.4.0



# Prepare TensorFlow CUDA operations

In this section, the TensorFlow operations implemented in CUDA (a programming language for GPUs from NVIDIA) are prepared to be used in the PointNet++ Jupyter notebooks.

**Note that this section of the notebook needs only be executed once to install and compile the TensorFlow CUDA operations.** But there should be no harm in executing it repeatedly. 

**Make sure that the file 'tf_ops.zip' is in the same folder as this notebook and all other notebooks you want to use these TensorFlow operations with.**

The following is commented out, assuming you already did this. If not, then uncomment and execute.

In [3]:
#!unzip -o -q "tf_ops.zip"

#!chmod u+x "tf_ops/compile_ops.sh"

#!tf_ops/compile_ops.sh

In [4]:
tf.random.set_seed(1234)

import numpy as np
import pandas as pd

# Helper Functions

The following helper function loads point cloud patches from csv files according to filenames with wildcards (*) from the same directory as Numpy arrays. For evaluation, there are also patches where the point indices are stored, so that the points in these patches can be matched again for a complete point cloud. If data_augmentation is True, then the patches are repeated and rotated by 10 different degrees. The dataset increases by a factor of 10, which results in much longer training times. But data augmentation also increases the accuracy by a few percent (maybe 2-3%). (If you compare it with a trained network without data augmentation, then you have to make sure you train with approximately the same number of batches, as one epoch with data augmentation is 10  times as large as one without.)

In [5]:
import glob

def load_data_from_csv(filepath, indices=False, data_augmentation=False):
        
    xyz_list = []
    feature_list = []
    labels_list = []
    indices_list = []
    
    for file in glob.glob(filepath):
        
        if indices:
            df = pd.read_csv(file, sep=' ', index_col=0)        
        else:
            df = pd.read_csv(file, sep=' ')        
            
        xyz = df[['x', 'y', 'z']].to_numpy()
       
        # make it vector based - START
        xmin = np.min(df['x'])
        xmax = np.max(df['x'])
        ymin = np.min(df['y'])
        ymax = np.max(df['y'])
        zmin = np.min(df['z'])
        zmax = np.max(df['z'])
                  
        # center by center point 
        xyz = xyz - np.array([0.5 * (xmax+xmin), 0.5 * (ymax+ymin), 0.5 * (zmax+zmin)])   
        
        xyz_list.append(xyz)
        feature_list.append(df[['intensity', 'return_number', 'number_of_returns']].to_numpy(dtype=float))
        labels_list.append(df['labels'].to_numpy())
        
        if indices:
            indices_list.append(df.index.to_numpy())
        
    xyz = np.stack(xyz_list)
    features = np.stack(feature_list)
    labels = np.stack(labels_list)
    
    if indices:
        idx = np.stack(indices_list)
    
    # rotate patches by their center point as data augmentation
    if data_augmentation:
                
        mat_list = []

        for i in range(0, 360, 36):
            rad = float(i) * np.pi / 180.0
            sin = np.sin(rad)
            cos = np.cos(rad)
            rot_mat = np.array([[cos, -sin, 0.0], [sin, cos, 0.0], [0.0, 0.0, 1.0]])
            mat_list.append(rot_mat)

        mat = np.stack(mat_list)
        mat.shape

        xyz = np.reshape(np.reshape(xyz, (-1, 3)) @ mat, (-1, 100000, 3))
        labels = np.tile(labels, (len(mat_list), 1))
        features = np.tile(features, (len(mat_list), 1, 1))
    
    xyz = np.concatenate((xyz, features), axis=-1)
    
    if indices:
        return xyz, labels[:, :, np.newaxis], idx
    else:
        return xyz, labels[:, :, np.newaxis]        

# Load ISPRS training patches

In [6]:
from pathlib import Path
import os

data_dir = str(Path.home()) + r'/coursematerial/GIS/ISPRS/PointNet++'
 
xyz, labels = load_data_from_csv(os.path.join(data_dir, 'patches', 'Vaihingen3D_Training*.csv'), data_augmentation=True)
    
print('X:', xyz.shape)
print('y:', labels.shape)

X: (420, 100000, 6)
y: (420, 100000, 1)


# Custom SA and FP Keras Layers

In the following, custom Keras layers for TensorFlow are implemented for the set abstraction layer and the feature propagation layer. 

**For an in-depth explanation of the implementation of the two layers, please refer to the respective code study notebooks.**

In [7]:
from tf_ops.tf_ops import (
    farthest_point_sample,
    gather_point,
    query_ball_point,
    group_point,
    knn_point,
    three_nn,
    three_interpolate
)

In [8]:
def sample_and_group(npoint, radius, nsample, xyz, points, knn=False, use_xyz=True):

    new_xyz = gather_point(xyz, farthest_point_sample(npoint, xyz)) # (batch_size, npoint, 3)

    if knn:
        _,idx = knn_point(nsample, xyz, new_xyz)
    else:
        idx, pts_cnt = query_ball_point(radius, nsample, xyz, new_xyz)
    grouped_xyz = group_point(xyz, idx) # (batch_size, npoint, nsample, 3)
    grouped_xyz -= tf.tile(tf.expand_dims(new_xyz, 2), [1,1,nsample,1]) # translation normalization
    if points is not None:
        grouped_points = group_point(points, idx) # (batch_size, npoint, nsample, channel)
        if use_xyz:
            new_points = tf.concat([grouped_xyz, grouped_points], axis=-1) # (batch_size, npoint, nample, 3+channel)
        else:
            new_points = grouped_points
    else:
        new_points = grouped_xyz

    return new_xyz, new_points, idx, grouped_xyz

In [9]:
from tensorflow.keras.layers import Layer

class Pointnet_SA(Layer):

    def __init__(
        self, npoint, radius, nsample, mlp, knn=False, use_xyz=True):

        super(Pointnet_SA, self).__init__()

        self.npoint = npoint
        self.radius = radius
        self.nsample = nsample
        self.mlp = mlp
        self.knn = False
        self.use_xyz = use_xyz

        self.mlp_list = []
        
    def build(self, input_shape):

        for i, n_filters in enumerate(self.mlp):
            self.mlp_list.append(keras.layers.Conv2D(n_filters, kernel_size=[1,1], activation='relu'))

        super(Pointnet_SA, self).build(input_shape)

    def call(self, xyz, points, training=True):

        if points is not None:
            if len(points.shape) < 3:
                points = tf.expand_dims(points, axis=0)

        new_xyz, new_points, idx, grouped_xyz = sample_and_group(
            self.npoint, self.radius, self.nsample, xyz, points, False, use_xyz=True)

        for i, mlp_layer in enumerate(self.mlp_list):
            new_points = mlp_layer(new_points, training=training)

        new_points = tf.math.reduce_max(new_points, axis=2, keepdims=True)

        return new_xyz, tf.squeeze(new_points)

In [10]:
class Pointnet_FP(Layer):

    def __init__(self, mlp):

        super(Pointnet_FP, self).__init__()

        self.mlp = mlp
        self.mlp_list = []


    def build(self, input_shape):

        for i, n_filters in enumerate(self.mlp):
            self.mlp_list.append(keras.layers.Conv2D(n_filters, kernel_size=[1,1], activation='relu'))

        super(Pointnet_FP, self).build(input_shape)

    def call(self, xyz1, xyz2, points1, points2, training=True):

        if points1 is not None:
            if len(points1.shape) < 3:
                points1 = tf.expand_dims(points1, axis=0)
        if points2 is not None:
            if len(points2.shape) < 3:
                points2 = tf.expand_dims(points2, axis=0)

        dist, idx = three_nn(xyz1, xyz2)
        dist = tf.maximum(dist, 1e-10)
        norm = tf.reduce_sum((1.0/dist),axis=2, keepdims=True)
        norm = tf.tile(norm,[1,1,3])
        weight = (1.0/dist) / norm
        interpolated_points = three_interpolate(points2, idx, weight)

        if points1 is not None:
            new_points1 = tf.concat(axis=2, values=[interpolated_points, points1]) # B,ndataset1,nchannel1+nchannel2
        else:
            new_points1 = interpolated_points
        new_points1 = tf.expand_dims(new_points1, 2)

        for i, mlp_layer in enumerate(self.mlp_list):
            new_points1 = mlp_layer(new_points1, training=training)

        new_points1 = tf.squeeze(new_points1)
        if len(new_points1.shape) < 3:
            new_points1 = tf.expand_dims(new_points1, axis=0)

        return new_points1

# Custom PointNet++ Model

Unfortunately, we cannot provide a detailed explanation of custom layers and models. But most of the code should already be clear or you can guess what is happening.

The neural network model with its layers is defined in **init_network()**, and **forward_pass()** is called in training and prediction, where the layers are called like in a functional model.

The hyperparameters (number of layers, samples points, radius, etc.) are empirically determined and follow rather common values used in aerial point cloud classification.

In this implementation, we also use the additional point cloud features like intensity, return number, etc. that are fed as features into the model (see l0_points that are input to the first SA layer and the last FP layer).

What is maybe unusual and unexpected is the use of dense layers after the feature propagation layers. As explained in the lecture, the class scores are the result form a multi-layer perceptron with shared weights that follows the last feature propagation layer. This multi-layer perceptron is applied on each point, and one would very likely expect a 2D convolutional layer. The explanation is simple: because the input tensor of the dense layer has more than 2 dimensions, the dense layer operates along the last axis, meaning it is executed as many times as there are elements in the second to last dimension, and performs the linear combination on the last dimension. You can verify the dimensions by uncommenting the print() method and build the model. It is basically a shared weight implementation of the multi-layer perceptron. Something that is often implemented by a convolutional layer.

As an addition side note that is worth mentioning: if the input tensor of a dense layer has too many dimensions, meaning it is not flattened, then the dense layer might not perform the operation that you might expect. Verify this by outputting the shape of the tensor that the dense layer produces.

In [32]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Dropout

class PointNet4ALSModel(Model):

    def __init__(self, batch_size, num_classes):
        
        super(PointNet4ALSModel, self).__init__()

        self.batch_size = batch_size
        self.keep_prob = 0.5
        self.num_classes = num_classes
        
        self.init_network()


    def init_network(self):
         
        self.sa_1 = Pointnet_SA(npoint=8192, radius= 1.0, nsample=16, mlp=[64, 64, 128])
        self.sa_2 = Pointnet_SA(npoint=4096, radius= 5.0, nsample=64, mlp=[128, 128, 256])
        self.sa_3 = Pointnet_SA(npoint=2048, radius=15.0, nsample=64, mlp=[128, 128, 256])

        self.fp_1 = Pointnet_FP(mlp = [256, 256])
        self.fp_2 = Pointnet_FP(mlp = [256, 256])
        self.fp_3 = Pointnet_FP(mlp = [128, 64])
        
        self.dense1 = Dense(128, activation='relu')
        self.dropout1 = Dropout(self.keep_prob)

        self.dense2 = Dense(self.num_classes, activation=tf.nn.softmax)
                    
    def forward_pass(self, input, training):

        # split the training data in xyz and features
        ln_xyz  = tf.slice(input, [0, 0, 0], [-1, -1,  3])    # point coordinates
        ln_feat = tf.slice(input, [0, 0, 3], [-1, -1, -1])    # point attributes
                
        l0_xyz = ln_xyz
        l0_points = ln_feat
    
        l1_xyz, l1_points = self.sa_1(l0_xyz, l0_points, training=training)
        l2_xyz, l2_points = self.sa_2(l1_xyz, l1_points, training=training)
        l3_xyz, l3_points = self.sa_3(l2_xyz, l2_points, training=training)

        l2_points = self.fp_1(l2_xyz, l3_xyz, l2_points, l3_points, training=training)
        l1_points = self.fp_2(l1_xyz, l2_xyz, l1_points, l2_points, training=training)
        l0_points = self.fp_3(l0_xyz, l1_xyz, l0_points, l1_points, training=training)

        # see explanation of why there is a dense layer here
#        print('Shape of input to dense layer:', l0_points.shape, '\n')
        
        net = self.dense1(l0_points)
        net = self.dropout1(net)
        pred = self.dense2(net)
                
        return pred

    def train_step(self, input):

        with tf.GradientTape() as tape:
            pred = self.forward_pass(input[0], True)
            loss = self.compiled_loss(input[1], pred)
            
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        self.compiled_metrics.update_state(input[1], pred)

        return {m.name: m.result() for m in self.metrics}


    def test_step(self, input):
               
        pred = self.forward_pass(input[0], False)
        loss = self.compiled_loss(input[1], pred)

        self.compiled_metrics.update_state(input[1], pred)

        return {m.name: m.result() for m in self.metrics}


    def call(self, input, training=False):

        return self.forward_pass(input, training)

In [33]:
model = PointNet4ALSModel(batch_size=1, num_classes=9)

model.build(input_shape=(1, 100000, 6))

print(model.summary())

Shape of input to dense layer: (1, 100000, 64) 

Model: "point_net4als_model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
pointnet_sa_12 (Pointnet_SA) multiple                  12928     
_________________________________________________________________
pointnet_sa_13 (Pointnet_SA) multiple                  66432     
_________________________________________________________________
pointnet_sa_14 (Pointnet_SA) multiple                  82816     
_________________________________________________________________
pointnet_fp_12 (Pointnet_FP) multiple                  197120    
_________________________________________________________________
pointnet_fp_13 (Pointnet_FP) multiple                  164352    
_________________________________________________________________
pointnet_fp_14 (Pointnet_FP) multiple                  41536     
______________________________________________________________

In [13]:
model.compile(
    optimizer=keras.optimizers.Adam(0.001),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()]
)

You should train PointNet++ with the provided data for 10 to 20 epochs. But be aware that each epoch takes almost 20 minutes.

But also for 5 epochs, you should already see quite good results.

Note that no validation data and metric is defined. It makes not much sense on this small dataset and the large overlaps of the patches. But in real-life scenario, one should always use validation data that is independent of the training data and the test data. So, basically we are training with one (or rather two) blind eye(s).

In [14]:
model.fit(
    x=xyz, 
    y=labels,
    batch_size=1,
    epochs = 5,
)

Epoch 1/5
420/420 [==============================] - 1024s 2s/step - loss: 0.9053 - sparse_categorical_accuracy: 0.6946
Epoch 2/5
420/420 [==============================] - 1025s 2s/step - loss: 0.4899 - sparse_categorical_accuracy: 0.8327
Epoch 3/5
420/420 [==============================] - 1025s 2s/step - loss: 0.3721 - sparse_categorical_accuracy: 0.8706
Epoch 4/5
420/420 [==============================] - 1025s 2s/step - loss: 0.3197 - sparse_categorical_accuracy: 0.8881
Epoch 5/5
420/420 [==============================] - 1024s 2s/step - loss: 0.2732 - sparse_categorical_accuracy: 0.9034


### Save model

You can save your model after training a few epochs for later use. Just uncomment the next cell.

In [15]:
#model.save_weights('./checkpoints/PointNet++')

### Load model

And load it at a later time. Just execute all cells above with the exception of the cells with model.fit() and to save the model. After loading, you can train for more epochs with the cell that contains the fit() method or continue on below.

In [16]:
#model.load_weights('./checkpoints/PointNet++')

# Evaluate with single patches

Evaluate the performance with the 42 patches of the evaluation dataset of ISPRS. Points in the overlap region will be evaluated repeatedly. 

In [17]:
xyz_val, labels_val = load_data_from_csv(os.path.join(data_dir, 'patches', 'Vaihingen3D_Evaluation*.csv'))
    
print(xyz_val.shape)
print(labels_val.shape)

(42, 100000, 6)
(42, 100000, 1)


Your accuracy should be close to (but probably below) 80%. (More like 78%.)

In [18]:
model.evaluate(xyz_val, labels_val, batch_size=1, verbose=1)

42/42 [==============================] - 93s 2s/step - loss: 0.6676 - sparse_categorical_accuracy: 0.7985


[0.6676174998283386, 0.7984952330589294]

# Combine and evaluate all patches

In the following, the patches are evaluated, then the results combines (with averaged class probabilities).

The following code is very rough and not suited for studying. It is meant to just execute and output the point clouds to study in CloudCompare.

### Load data

In [19]:
xyz_val, labels_val, indices_val = load_data_from_csv(os.path.join(data_dir, 'patchesIndexed', 'Vaihingen3D_Evaluation*.csv'), indices=True)
    
print(xyz_val.shape)
print(labels_val.shape)
print(indices_val.shape)

(42, 100000, 6)
(42, 100000, 1)
(42, 100000)


### Combine

In [20]:
predictions = model.predict(xyz_val, batch_size=1, verbose=1)
prd = np.reshape(predictions, (-1, 9))
idx = indices_val.flatten()
idx_arr = np.argsort(idx)
prd_sorted = np.take(prd, idx_arr, axis=0)
idx_arr_sorted = np.take(idx, idx_arr)
u, indices, counts = np.unique(idx_arr_sorted, return_index=True, return_counts=True)
added_pred = np.add.reduceat(prd_sorted, indices)
pred_avg = added_pred / np.repeat(counts[:, np.newaxis], 9, axis=1)
pred_labels_from_avg = np.argmax(pred_avg, axis=1)
u, counts = np.unique(pred_labels_from_avg, return_counts=True)

42/42 [==============================] - 93s 2s/step


### Load original points

In [21]:
pn_dir = str(Path.home()) + r'/coursematerial/GIS/ISPRS/PointNet'

column_names = ['x', 'y', 'z', 'intensity', 'return_number', 'number_of_returns', 'labels']
        
df_isprs_val = pd.read_csv(os.path.join(pn_dir, 'Vaihingen3D_Evaluation.pts'), sep=' ', names=column_names, header=None)

### Classification report

You should see a better accuracy with the merged patches. Accuracy should be a little higher than 80%. (If you adapted the model and got more than 83%, then please tell us what you did!!!)

In [22]:
from sklearn.metrics import classification_report
class_names = ['Powerline', 'Low vegetation', 'Impervious surfaces', 'Car', 'Fence/Hedge', 'Roof', 'Facade', 'Shrub', 'Tree']
print(classification_report(df_isprs_val['labels'], pred_labels_from_avg, target_names=class_names))

                     precision    recall  f1-score   support

          Powerline       0.86      0.54      0.66       600
     Low vegetation       0.84      0.80      0.82     98690
Impervious surfaces       0.86      0.95      0.90    101986
                Car       0.88      0.37      0.52      3708
        Fence/Hedge       0.39      0.15      0.22      7422
               Roof       0.88      0.94      0.91    109048
             Facade       0.55      0.49      0.52     11224
              Shrub       0.41      0.30      0.35     24818
               Tree       0.76      0.78      0.77     54226

           accuracy                           0.82    411722
          macro avg       0.71      0.59      0.63    411722
       weighted avg       0.80      0.82      0.81    411722



### Confusion matrix

In [23]:
from sklearn.metrics import confusion_matrix

class_names = ['Powerline', 'Low vegetation', 'Impervious surfaces', 'Car', 'Fence/Hedge', 'Roof', 'Facade', 'Shrub', 'Tree']

cm = confusion_matrix(df_isprs_val['labels'], pred_labels_from_avg)

df = pd.DataFrame(data=cm, columns=class_names, index=class_names)
df

,Powerline,Low vegetation,Impervious surfaces,Car,Fence/Hedge,Roof,Facade,Shrub,Tree
Powerline,323,1,0,0,0,236,6,14,20
Low vegetation,0,78538,12369,46,340,1948,1174,3426,849
Impervious surfaces,0,4561,96495,11,1,764,39,111,4
Car,0,403,859,1372,345,231,48,448,2
Fence/Hedge,0,1005,463,71,1133,560,64,3144,982
Roof,17,1052,239,0,19,102753,1090,576,3302
Facade,20,1024,308,2,13,2569,5543,538,1207
Shrub,5,5865,721,62,799,2453,553,7503,6857
Tree,10,1404,111,4,289,5622,1647,2741,42398


### Output colorized point cloud

In [24]:
def colored_point_cloud(xyz, y, filename):

    color_map = np.array([
        [255, 255, 125],
        [  0, 255, 255],
        [255, 255, 255],
        [255, 255,   0],
        [  0, 255, 125],
        [  0,   0, 255],
        [  0, 125, 255],
        [125, 255,   0],
        [  0, 255,   0]])
    
    u, inverses = np.unique(y, return_inverse=True)    
    
    colors = color_map[inverses]
    
    r_s = pd.Series(data=colors[:,0], name='red')
    g_s = pd.Series(data=colors[:,1], name='green')
    b_s = pd.Series(data=colors[:,2], name='blue')

    df = pd.DataFrame(xyz, columns=['x', 'y', 'z'])    

    df['red'] = r_s
    df['green'] = g_s
    df['blue'] = b_s
    
    df.to_csv(filename, index=False, header=False)

In [25]:
colored_point_cloud(df_isprs_val[['x', 'y', 'z']], pred_labels_from_avg, 'CombinedResult.csv')

# For expert students with too much time:

The ISPRS datasets also contains aerial images. One could overlay the aerial images with the point clouds (using e.g. LAStools) to also have color information as additional input features. If someone is interested, we could provide some guidance for the point cloud and image fusion using LAStools, and we could provide the rough code for the generation of the patches.